In [ ]:
import sys
import os

from HelmholtzSolver2D import *
from ExampleBoundaries import Square

In [ ]:
c         = 344.0 # speed of sound [m/s]
rho       = 1.205 # density of air [kg/m^3]
frequency = 400.0 # frequency [Hz]


# Test Problem 1
# Dirichlet boundary condition with phi = sin(k/sqrt(2)*x) * sin(k/sqrt(2)*y)
#
solver = HelmholtzSolver2D(*(Square()))

boundaryCondition = BoundaryCondition(solver.aElement.shape[0])
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)

k = frequencyToWavenumber(frequency)

boundaryCondition.f[:] = np.sin(k/np.sqrt(2.0) * solver.aCenters[:,0]) \
                       * np.sin(k/np.sqrt(2.0) * solver.aCenters[:,1])

boundaryIncidence = BoundaryIncidence(solver.aElement.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

interiorPoints = np.array([[0.0250, 0.0250],
                           [0.0750, 0.0250],
                           [0.0250, 0.0750],
                           [0.0750, 0.0750],
                           [0.0500, 0.0500]], dtype=np.float32)

interiorIncidentPhi = np.zeros(interiorPoints.shape[0], dtype=np.complex64)

boundarySolution = solver.solveInteriorBoundary(k, boundaryCondition, boundaryIncidence)
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print("Test Problem 1")
print("==============\n")
print(boundarySolution)
printInteriorSolution(boundarySolution, interiorPhi)


# Test Problem 2
# von Neumann boundary condition such that phi = sin(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# Differentiate with respect to x and y to obtain outward normal:
# dPhi/dX = k/sqrt(2) * cos(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# dPhi/dY = k/sqrt(2) * sin(k/sqrt(2) * x) * cos(k/sqrt(2) * y)
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
c = k / np.sqrt(2.0)
for i in range(solver.aCenters.shape[0]):
    x = solver.aCenters[i, 0]
    y = solver.aCenters[i, 1]
    if (x < 1e-7):
        boundaryCondition.f[i] = -c * np.cos(c * x) * np.sin(c * y)
    elif (x > 0.1 - 1e-7):
        boundaryCondition.f[i] =  c * np.cos(c * x) * np.sin(c * y)
    elif (y < 1e-7):
        boundaryCondition.f[i] = -c * np.sin(c * x) * np.cos(c * y)
    else:
        boundaryCondition.f[i] =  c * np.sin(c * x) * np.cos(c * y)        

boundarySolution = solver.solveInteriorBoundary(k, boundaryCondition, boundaryIncidence)
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print("\n\nTest Problem 2")
print("==============\n")
print(boundarySolution)
printInteriorSolution(boundarySolution, interiorPhi)

   
# Test Problem 3
# The test problem computes the field produced by a unit source at
# the point (0.5,0.25) within the square with a rigid boundary.
# The rigid boundary implies the bondary condition v=0.
# The test problem computes the field produced by a unit source at
# the point (0.5,0.25) within the square with a rigid boundary.
# The incident velocity potential is given by {\phi}_inc=i*h0(kr)/4
# where r is the distance from the point (0.5,0.25)
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
boundaryCondition.f.fill(0.0)

p = np.array([0.05, 0.025], dtype=np.float32)
for i in range(solver.aCenters.shape[0]):
    r = solver.aCenters[i] - p
    R = norm(r)
    boundaryIncidence.phi[i] = 0.25j * hankel1(0, k * R)
    if solver.aCenters[i, 0] < 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * (-r[0] / R)
    elif solver.aCenters[i, 0] > 0.1 - 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * ( r[0] / R)
    elif solver.aCenters[i, 1] < 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * (-r[1] / R)
    elif solver.aCenters[i, 1] > 0.1 - 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * ( r[1] / R)
    else:
        assert False, "All cases must be handled above."
        
for i in range(interiorIncidentPhi.size):
    r = interiorPoints[i] - p
    R = norm(r)
    interiorIncidentPhi[i] = 0.25j * hankel1(0, k * R)
       
boundarySolution = solver.solveInteriorBoundary(k, boundaryCondition, boundaryIncidence)
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print("\n\nTest Problem 3")
print("==============\n")
print(boundarySolution)
printInteriorSolution(boundarySolution, interiorPhi)

Test Problem 1

Density of medium:      1.205 kg/m^3
Speed of sound:         344.0 m/s
Wavenumber (Frequency): 7.306029426953008 (400.0 Hz)

index          Potential                   Pressure                    Velocity              Intensity
    1   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -1.6414e-01+  6.9431e-03i     0.0000e+00
    2   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -5.0024e-01+  8.1575e-03i     0.0000e+00
    3   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -8.3281e-01+  9.1901e-03i     0.0000e+00
    4   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -1.1621e+00+  1.0149e-02i     0.0000e+00
    5   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -1.4863e+00+  1.1014e-02i     0.0000e+00
    6   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -1.8031e+00+  1.1630e-02i     0.0000e+00
    7   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -2.1077e+00+  1.1436e-02i     0.0000e+00
    8   0.0000e+00+  0.0000e

Copyright (C) 2017 Frank Jargstorff

This file is part of the AcousticBEM library.

AcousticBEM is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

AcousticBEM is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with AcousticBEM. If not, see http://www.gnu.org/licenses/.